NOTE: because of the way remapping works for TMAs, we recommend running this notebook cell-by-cell rather than all at once.

In [ ]:
import json
import os
from skimage.io import imread

from ark.mibi import tiling_utils

# suppress mpl deprecation
import warnings
from matplotlib.cbook import mplDeprecation
warnings.filterwarnings("ignore", category=mplDeprecation)

# 1. Automatically generate tiled regions

This automatically creates tiles which define FOVs on the MIBI slide based on user-set parameters.

### Define paths to the JSON data

You'll need to define the following:

* `base_dir`: the root directory of the tiling data
* `json_tiling_dir`: the directory in `base_dir` containing the information to read and write the FOV info
* `fov_list_path`: the list of FOVs to use for tiling
* `moly_path`: the path to the Moly point, needed if you want to insert this between FOVs and/or runs

In [ ]:
base_dir = "../data/example_dataset"
json_tiling_dir = os.path.join(base_dir, "json_tiling_data")
fov_list_path = os.path.join(json_tiling_dir, 'fov_list_single_fov_tma.json')
moly_path = os.path.join(json_tiling_dir, 'moly_point.json')

### Define whether TMA is used or not

In [ ]:
tma = True

### Define the path to store the generated FOVs

In [ ]:
fov_output_path = os.path.join(json_tiling_dir, 'fov_output_tma.json')

### Set tiling parameters

The following parameters will be defined:

* Starting x coordinate and y coordinate for each fov (taken from `fov_list_path`)
* Number of fovs along the x- and y-axis for each fov (set by user)
* x- and y-axis step size (set by user)
* Whether to randomize the order of the fovs or not (set by user)

Notes about MIBI coordinate axes:

* The x-axis goes from left to right ascending
* The y-axis goes from bottom to top ascending

Note for TMAs:

* A region is defined by the upper-left and the bottom-right corner FOV. FOVs are generated in between based on the parameters specified.
* The upper-left corner is expected to be listed before the bottom-right corner FOV for a region
* If multiple regions are specified in the notebook, one region's upper-left and bottom-right corner FOV are expected to be defined before another's

In [ ]:
tiling_params, moly_point = tiling_utils.set_tiling_params(
    fov_list_path,
    moly_path,
    tma=tma
)

### Create the FOV tiles for each region

In [ ]:
auto_fov_regions = tiling_utils.generate_fov_list(
    tiling_params,
    moly_point,
    tma=tma
)

In [ ]:
# write the tiled output
with open(fov_output_path, 'w') as fop:
    json.dump(auto_fov_regions, fop)

# 2. Slide overlay QC (for TMAs only)

TODO: separate this to a different notebook.

Using a pre-defined JSON of FOVs, compares with the automatically-generated FOVs from the previous step and allows for remapping if necessary.

### Define QC parameters, read data in

Define the following parameters:

* `manual_fov_output_path`: the path to your proposed set of FOVs
* `slide_path`: the path to the slide which to take the FOVs
* `mapping_path`: the path which to save the final mapping of your proposed FOVs to the FOVs generated by the script (contained in `augo_fov_regions`)
* `remapped_fov_path`: the path to write the tiles with remapped FOV names

NOTE: the data contained in `manual_fov_output_path` is assumed to have the same FOV dimensions and the same Moly point as the data in `fov_output_path`.

In [ ]:
manual_fov_output_path = os.path.join(json_tiling_dir, 'noah_test_tiles.json')
slide_path = os.path.join(json_tiling_dir, 'noah_test_slide.png')
mapping_path = os.path.join(json_tiling_dir, 'manual_auto_map.json')
remapped_fov_path = os.path.join(json_tiling_dir, 'noah_test_tiles_remapped.json')

In [ ]:
# load the proposed set of tiles in
with open(manual_fov_output_path, 'r') as mfop:
    manual_fov_regions = json.load(mfop)

In [ ]:
# load the slide image in
slide_data = imread(slide_path)

### Map proposed tiles to their closest automatically-generated tile (Euclidean)

In [ ]:
manual_to_auto_map, manual_fovs_info, auto_fovs_info = tiling_utils.assign_closest_fovs(
    manual_fov_regions,
    auto_fov_regions,
    moly_point['name']
)

### Visualize and remap tiles

Usage notes:

* Proposed FOVs are drawn in red. Automatically-generated FOVs are drawn in blue.
* The selected proposed FOV and its mapped automatically-generated FOV are colored a darker shade.
* The `Manually-defined FOV` tab can be used to visualize current mappings to automatically-generated FOVs.
* The `Automatically-generated FOV` tab is used for re-mapping a FOV created by this script to one of yours.
* When you're done generating your desired mapping, click the `Save mapping`. This will save your mapping to `mapping_path` specified earlier.

Note:

* The cells after this interactive widget (in the section `Use mapping to rename FOVs in tiled_regions_proposed`) are only to be run after you're satisfied with the mapping and clicked `Save mapping`.
* Prior to clicking `Save mapping` and re-running the cells in that section, ignore any error messages that may appear there.

In [12]:
%matplotlib widget
tiling_utils.interactive_remap(
    manual_to_auto_map,
    manual_fovs_info,
    auto_fovs_info,
    slide_data,
    mapping_path,
    draw_radius=7,
    figsize=(7, 7)
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Output()

### Use mapping to rename FOVs in `manual_fov_regions`

In [ ]:
# load the mapping saved in the interactive visualization in
with open(mapping_path, 'r') as mp:
    mapping = json.load(mp)

In addition to renaming FOVs in `manual_fov_regions`, the following steps can be done if specified:

* The FOVs are randomized (set `randomize = True` in the following cell). 
* Moly points are inserted at a specified interval (set `insert_moly = True` and `moly_interval` to the desired value in the following cell). The same Moly point returned by `set_tiling_params` in step 1 is used.

In [ ]:
# whether to randomize the FOVs in remapped_fov_regions
randomize = False

# whether to insert Moly points between a specified interval of FOVs in remapped_fov_regions
moly_insert = False
moly_interval = 5

In [ ]:
# rename FOVs, randomize the order, and insert Moly points at a specified interval
remapped_fov_regions = tiling_utils.remap_and_reorder_fovs(
    manual_fov_regions,
    mapping,
    moly_point,
    randomize=randomize,
    moly_insert=moly_insert,
    moly_interval=moly_interval
)

In [ ]:
# save remapped_fov_regions with the new FOV names (and randomized and with Moly points inserted if specified)
with open(remapped_fov_path, 'w') as rtp:
    json.dump(remapped_fov_regions, rtp)